In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
NDVI_PATH = "../data/PROCESSED/ndvi.csv"
PROD_PATH = "../data/PROCESSED/manhuacu.csv"

In [ ]:
# LSTM Hyperparameters
LSTM_WINDOW_SIZE = 20
LSTM_HIDDEN_SIZE = 64
LSTM_NUM_LAYERS = 2
LSTM_DROPOUT = 0.2
LSTM_EPOCHS = 200

In [ ]:
def get_day_of_year_index(date: datetime):
    """Convert date to day of year."""
    return datetime(date.year, date.month, date.day).timetuple().tm_yday - 1


def get_sin_cos(x: float):
    """Convert x to sin and cos."""
    rad = 2 * np.pi * x
    return (np.sin(rad), np.cos(rad))


def encode_date(date: datetime):
    is_leap_year = 1 if date.year % 4 == 0 else 0
    total_year_days = 366 if is_leap_year else 365

    day_index = get_day_of_year_index(date)
    
    frac = day_index / total_year_days
    return get_sin_cos(frac)

# Test
print("Encoding date 2020-01-01")
print(encode_date(datetime(2020, 1, 1)))  # (0.0, 1.0)
print("\n")
print("Encoding date 2020-06-01")
print(encode_date(datetime(2020, 6, 1)))  # (0.5, 0.0)
print("\n")
print("Encoding date 2020-12-31")
print(encode_date(datetime(2020, 12, 31)))  # (0.9999999999999999, 1.0)
print("\n")

## 1. Load Data

In [ ]:

NDVI = pd.read_csv(NDVI_PATH)

NDVI["N_Observations"] = NDVI.groupby("Year")["Data"].transform("count")

NDVI[["Date_sin", "Date_cos"]] = NDVI["Data"].apply(
    lambda x: pd.Series(encode_date(datetime.strptime(x, "%Y-%m-%d")))
)

# Assert order by Data (ascending)
NDVI = NDVI.sort_values(by="Data", ascending=True)

NDVI = NDVI[(NDVI["Year"] >= 2000) & (NDVI["Year"] <= 2024)]

NDVI

In [ ]:
PROD = pd.read_csv(PROD_PATH)
PROD = PROD[(PROD["Year"] >= 2000) & (PROD["Year"] <= 2024)]
max_productivity = PROD["Productivity (kg/ha)"].max()
PROD["Normalized_productivity"] = PROD["Productivity (kg/ha)"] / max_productivity
PROD

## 2. Prepare Data

In [ ]:
NDVI_last_20_per_year = NDVI.groupby("Year").tail(20)
NDVI_last_20_per_year

In [ ]:
# Plotting NDVI
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(
    NDVI_last_20_per_year["Data"],
    NDVI_last_20_per_year["NDVI"],
    label="NDVI",
    color="green",
)
plt.title("NDVI over the last 20 days of each year")

plt.xlabel("Year")
plt.xticks(
    ticks=NDVI_last_20_per_year.groupby("Year").tail(1).index,
    labels=NDVI_last_20_per_year.groupby("Year").tail(1)["Year"],
)
plt.ylabel("NDVI")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
class MLPDataset(torch.utils.data.Dataset):
    def __init__(self, ndvi_df, prod_df):
        self.ndvi_df = ndvi_df
        self.prod_df = prod_df

    def __len__(self):
        return self.ndvi_df["Year"].nunique()

    def __getitem__(self, idx):
        years = self.ndvi_df["Year"].sort_values().unique()
        if idx >= len(years):
            raise IndexError("Index out of range")
        year = years[idx]
        ndvi = self.ndvi_df[self.ndvi_df["Year"] == year]["NDVI"].values
        prod = self.prod_df[self.prod_df["Year"] == year][
            "Productivity (kg/ha)"
        ].values[0]
        return torch.tensor(ndvi, dtype=torch.float32), torch.tensor(
            prod, dtype=torch.float32
        )


dataset = MLPDataset(NDVI_last_20_per_year, PROD)
dataset[0]

train_size = int(0.8 * len(dataset))
valid_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset, valid_dateset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, valid_size, test_size]
)

## 3. Model training

### 3.1. Multi-layer Perceptron

Essa rede é uma feedforward perceptron multi-layer comum (1 camada interna).

As entradas são os 20 últimos NDVIs do ano, a saída é a produtividade prevista (kg/ha).

In [ ]:
mlp_network = nn.Sequential(
    nn.Linear(20, 128),
    nn.ReLU(),
    nn.Linear(128, 256),
    nn.ReLU(),
    nn.Linear(256, 1)
)
mlp_network = mlp_network.to(device)
optimizer = optim.Adam(mlp_network.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

losses = []
best_loss = float("inf")
for i in range(100):
    mlp_network.train()
    for ndvi, prod in train_dataset:
        ndvi, prod = ndvi.to(device), prod.to(device)
        optimizer.zero_grad()
        pred = mlp_network(ndvi)
        loss = loss_fn(pred, torch.tensor([prod], device=device))
        loss.backward()
        optimizer.step()
    
    epoch_losses = []
    mlp_network.eval()
    with torch.no_grad():
        for ndvi, prod in valid_dateset:
            ndvi, prod = ndvi.to(device), prod.to(device)
            pred = mlp_network(ndvi)
            loss = loss_fn(pred, torch.tensor([prod], device=device))
            epoch_losses.append(loss.item())
            
        if np.mean(epoch_losses) < best_loss:
            best_loss = np.mean(epoch_losses)
            torch.save(mlp_network.state_dict(), "mlp.pth")
            print(f"Saving MLP model\tepoch: {i+1}\tvalidation loss: {best_loss:.4f}")
    
    losses.append(np.mean(epoch_losses))

In [ ]:
def plot_training_loss(losses):
    """
    Plots the training loss over epochs.

    Args:
        losses (list): List of loss values for each epoch.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(losses, label="Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Epochs")
    plt.legend()
    plt.grid(True)
    plt.show()
    
plot_training_loss(losses)

### 3.2. LSTM

In [ ]:
class LSTMDataset(torch.utils.data.Dataset):
    def __init__(self, ndvi_df, prod_df, window_size=LSTM_WINDOW_SIZE):
        self.ndvi_df = ndvi_df
        self.prod_df = prod_df
        self.window_size = window_size

    def __len__(self):
        return len(self.ndvi_df) - self.window_size

    def __getitem__(self, idx):
        ndvi = self.ndvi_df.iloc[idx : idx + self.window_size][
            ["NDVI", "Date_sin", "Date_cos"]
        ].values

        year = self.ndvi_df.iloc[idx + self.window_size]["Year"]
        prod = self.prod_df[self.prod_df["Year"] == year][
            "Productivity (kg/ha)"
        ].values[0]

        return torch.tensor(ndvi, dtype=torch.float32), torch.tensor(
            prod, dtype=torch.float32
        )



In [ ]:
# Test Dataset
dataset = LSTMDataset(NDVI_last_20_per_year, PROD, LSTM_WINDOW_SIZE)

train_size = int(0.8 * len(dataset))
valid_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - valid_size
train_dataset = torch.utils.data.Subset(dataset, range(train_size))
valid_dateset = torch.utils.data.Subset(
    dataset, range(train_size, train_size + valid_size)
)
test_dataset = torch.utils.data.Subset(
    dataset, range(train_size + valid_size, train_size + valid_size + test_size)
)

In [ ]:
import torch.nn as nn
import torch.optim as optim
import numpy as np


# Define model with Linear layer
class LSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, dropout=LSTM_DROPOUT
        )
        self.fc = nn.Linear(hidden_size, 1)  # Output a single value

    def forward(self, x, hidden_n=None, hidden_c=None):
        if hidden_n is None or hidden_c is None:
            out, _ = self.lstm(x)
            return self.fc(out[:, -1, :])  # Get output of last time step
        else:
            out, (hidden_n, hidden_c) = self.lstm(x, (hidden_n, hidden_c))
            out = self.fc(out[:, -1, :])  # Get output of last time step
            return out, (hidden_n, hidden_c)


lstm_model = LSTMRegressor(
    input_size=3, hidden_size=LSTM_HIDDEN_SIZE, num_layers=LSTM_NUM_LAYERS
).to(device)
optimizer = optim.Adam(lstm_model.parameters())
loss_fn = nn.MSELoss()

lstm_losses = []
best_loss = float("inf")
saved_epoch = 0


for i in range(LSTM_EPOCHS):
    h_n = torch.zeros(LSTM_NUM_LAYERS, 1, LSTM_HIDDEN_SIZE).to(device)  # Hidden state
    h_c = torch.zeros(LSTM_NUM_LAYERS, 1, LSTM_HIDDEN_SIZE).to(device)  # Cell state

    lstm_model.train()
    for ndvi, prod in train_dataset:
        ndvi, prod = ndvi.to(device), prod.to(device)
        optimizer.zero_grad()

        pred, (h_n, h_c) = lstm_model(
            ndvi.unsqueeze(0), h_n.detach(), h_c.detach()
        )  # Add batch dimension: [1, seq_len, 3]
        # print(pred.shape)
        last_pred = pred[:, -1]  # Get the last prediction
        # print(last_pred.shape)
        loss = loss_fn(last_pred, prod.unsqueeze(0))  # Match shape: [1]
        loss.backward()
        optimizer.step()

    epoch_losses = []
    lstm_model.eval()
    with torch.no_grad():
        for ndvi, prod in valid_dateset:
            ndvi, prod = ndvi.to(device), prod.to(device)
            pred = lstm_model(ndvi.unsqueeze(0))
            last_pred = pred[:, -1]  # Get the last prediction
            loss = loss_fn(last_pred, prod.unsqueeze(0))
            epoch_losses.append(loss.item())

        avg_loss = np.mean(epoch_losses)
        if avg_loss < best_loss:
            best_loss = avg_loss
            saved_epoch = i + 1
            torch.save(lstm_model.state_dict(), "lstm.pth")

    # if (i) % 10 == 0:
    print(f"Epoch {i+1}/{LSTM_EPOCHS} - Loss: {avg_loss:.4f}")
    lstm_losses.append(avg_loss)

print(f"\n\nSaved LSTM model\tepoch: {saved_epoch}\tvalidation loss: {best_loss:.4f}")

In [ ]:
plot_training_loss(lstm_losses)